#### Configurando o ambiente.

In [1]:
!pip install pyspark
!pip install -u plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 41 kB/s 
     |████████████████████████████████| 199 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=80bffd8196b8d9f290c6d2b20c6cc68771f43200d8b2287d12960d3fb1a90688
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -u


In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff


In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
.builder \
.appName("Python Spark create RDD example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate

In [4]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as f
from IPython.core.display import HTML
from pyspark.sql.functions import col, asc,desc
from pyspark.sql.functions import regexp_replace

# melhor visualização do dataframe
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Session builder
spark = SparkSession.builder.appName('sparkdf').getOrCreate()

#### Importação das tabelas de registro de Covid no Brasil.

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
caminho_local = 'gdrive/MyDrive/Pyspark/Projeto_covid'

Mounted at /content/gdrive


In [6]:
covid_2020_1 = spark.read.format('csv')\
                         .option("sep",";",)\
                         .options(header='true',\
                             inferschema='true')\
                    .load(f'{caminho_local}/HIST_PAINEL_COVIDBR_2020_Parte1_09ago2022.csv',header=True)

In [7]:
covid_2020_2 = spark.read.format('csv')\
                         .option("sep",";",)\
                         .options(header='true',\
                             inferschema='true')\
                    .load(f'{caminho_local}/HIST_PAINEL_COVIDBR_2020_Parte2_09ago2022.csv',header=True)

In [8]:
covid_2021_1 = spark.read.format('csv')\
                         .option("sep",";",)\
                         .options(header='true',\
                             inferschema='true')\
                    .load(f'{caminho_local}/HIST_PAINEL_COVIDBR_2021_Parte1_09ago2022.csv',header=True)

In [9]:
covid_2021_2 = spark.read.format('csv')\
                         .option("sep",";",)\
                         .options(header='true',\
                                  inferschema='true')\
                    .load(f'{caminho_local}/HIST_PAINEL_COVIDBR_2021_Parte2_09ago2022.csv',header=True)

In [10]:
covid_2022_1 = spark.read.format('csv')\
                         .option("sep",";",)\
                         .options(header='true',\
                                  inferschema='true')\
                    .load(f'{caminho_local}/HIST_PAINEL_COVIDBR_2022_Parte1_09ago2022.csv',header=True)

In [11]:
covid_2022_2 = spark.read.format('csv')\
                         .option("sep",";",)\
                         .options(header='true',\
                                  inferschema='true')\
                    .load(f'{caminho_local}/HIST_PAINEL_COVIDBR_2022_Parte2_09ago2022.csv',header=True)

#### Foi feita a união das tabelas para melhor visualização.

In [12]:
covid = covid_2020_1.union(covid_2020_1)\
                    .union(covid_2020_2)\
                    .union(covid_2021_1)\
                    .union(covid_2021_2)\
                    .union(covid_2022_1)\
                    .union(covid_2022_2)

covid.show()

+------+------+---------+-----+------+--------------+---------------+-------------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|               data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|interior/metropolitana|
+------+------+---------+-----+------+--------------+---------------+-------------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|Brasil|  null|     null|   76|  null|          null|           null|2020-02-25 00:00:00|        9|       210147125|           0.0|         0|              0|          0|               0|                    0|                  null|
|Brasil|  null|     null|   76|  null|          null|           null

#### Alteração do formato da coluna data.

In [13]:
covid = covid.withColumn("data",\
                          f.date_format("data", "yyyy-MM-dd"))
covid.show()

+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|interior/metropolitana|
+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+----------------------+
|Brasil|  null|     null|   76|  null|          null|           null|2020-02-25|        9|       210147125|           0.0|         0|              0|          0|               0|                    0|                  null|
|Brasil|  null|     null|   76|  null|          null|           null|2020-02-26|        9|       2101471

#### Contagem da quantidade de registros na tabela.

In [14]:
covid.count()

5581403

In [15]:
covid.select("interior/metropolitana").distinct().show()

+----------------------+
|interior/metropolitana|
+----------------------+
|                  null|
|                     1|
|                     0|
+----------------------+



#### Devido a coluna interior/metropolitana não evidenciar quando a localidade se refere ao interior ou a área metropolitana, a mesma foi removida


In [16]:
covid = covid.drop("interior/metropolitana")

####Foi separado o dataframe do Brasil para verificar os casos e obitos novos referentes ao país, desconsiderando as regiões, estados e municipios.

In [17]:
brasil = covid.select("data", "regiao", "casosNovos","obitosNovos")\
              .filter(f.col("regiao") == "Brasil")\
              .sort(col("casosNovos").desc(), col("obitosNovos").desc())

brasil.show()

+----------+------+----------+-----------+
|      data|regiao|casosNovos|obitosNovos|
+----------+------+----------+-----------+
|2022-02-03|Brasil|    298408|       1041|
|2022-01-28|Brasil|    269968|        799|
|2022-01-27|Brasil|    228954|        672|
|2022-01-26|Brasil|    224567|        570|
|2022-01-19|Brasil|    204854|        338|
|2022-02-05|Brasil|    197442|       1308|
|2022-02-01|Brasil|    193465|        929|
|2022-02-04|Brasil|    184311|        493|
|2022-01-25|Brasil|    183722|        487|
|2022-01-29|Brasil|    179816|        640|
|2022-02-09|Brasil|    178814|       1264|
|2022-02-08|Brasil|    177027|       1189|
|2022-02-02|Brasil|    172903|        893|
|2022-01-20|Brasil|    168495|        350|
|2022-01-21|Brasil|    166539|        358|
|2022-02-11|Brasil|    166009|       1135|
|2022-02-10|Brasil|    164066|        943|
|2022-01-22|Brasil|    157393|        238|
|2021-09-18|Brasil|    150106|        935|
|2022-02-16|Brasil|    147734|       1085|
+----------

####Foi separado o dataframe das Regiões para verificar os casos e obitos novos referentes as regiões, desconsiderando o país , estados e municipios.

In [18]:
regiao = covid.select( "regiao", "casosNovos","obitosNovos")\
              .filter(f.col("regiao") != "Brasil")

regiao.sort(col("casosNovos").desc(), col("obitosNovos").desc()).show()

+--------+----------+-----------+
|  regiao|casosNovos|obitosNovos|
+--------+----------+-----------+
| Sudeste|    105700|        371|
|   Norte|     74352|        727|
|   Norte|     73235|        719|
| Sudeste|     69223|          7|
|     Sul|     64036|         62|
|Nordeste|     59430|          0|
|Nordeste|     49132|          0|
|     Sul|     47815|       1134|
|     Sul|     47101|       1108|
|     Sul|     45154|         90|
|     Sul|     45020|        247|
|     Sul|     43776|       1043|
|     Sul|     43028|       1020|
|Nordeste|     42771|          3|
|Nordeste|     42547|         64|
| Sudeste|     40753|         44|
|Nordeste|     40290|         16|
|     Sul|     40170|        177|
| Sudeste|     39271|        109|
| Sudeste|     37611|        349|
+--------+----------+-----------+
only showing top 20 rows



Foi separado o dataframe dos Estados para verificar os casos e obitos novos referentes a cada estado, desconsiderando as país, regiões e municipios.

In [19]:
estados = covid.select("data", "regiao", "estado", "casosNovos", "obitosNovos")\
               .filter((f.col("regiao") != "Brasil")&(f.col("municipio").isNull()))\
               .orderBy(col("obitosNovos").desc())\
   

estados.show()

+----------+-------+------+----------+-----------+
|      data| regiao|estado|casosNovos|obitosNovos|
+----------+-------+------+----------+-----------+
|2021-04-06|Sudeste|    SP|     22794|       1389|
|2021-04-08|Sudeste|    SP|     21004|       1299|
|2021-04-13|Sudeste|    SP|     18397|       1282|
|2021-03-30|Sudeste|    SP|     21360|       1209|
|2021-03-26|Sudeste|    SP|     21489|       1193|
|2021-03-31|Sudeste|    SP|     23169|       1160|
|2021-04-20|Sudeste|    SP|     19060|       1122|
|2021-04-14|Sudeste|    SP|     18790|       1095|
|2021-04-01|Sudeste|    SP|     26567|       1082|
|2021-04-15|Sudeste|    SP|     18067|       1060|
|2021-03-27|Sudeste|    SP|     18124|       1051|
|2021-04-27|Sudeste|    SP|     17992|       1044|
|2021-03-23|Sudeste|    SP|     20942|       1021|
|2021-04-09|Sudeste|    SP|     20701|       1008|
|2021-04-21|Sudeste|    SP|     17123|        977|
|2021-05-25|Sudeste|    SP|     17477|        898|
|2021-06-15|Sudeste|    SP|    

#### Foi realizada uma exploração dos dados para verificar se as colunas casosAcumulado e obitosAcumulado estão consistentes. Foi utilizado como amostra os estados de São Paulo e Rio de Janeiro.

In [20]:
covid_sp = covid.filter(f.col("estado") == "SP")\
                .orderBy(col("data").desc())

covid_sp.show()

+-------+------+-------------------+-----+------+--------------+--------------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
| regiao|estado|          municipio|coduf|codmun|codRegiaoSaude|     nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|
+-------+------+-------------------+-----+------+--------------+--------------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
|Sudeste|    SP|    Álvares Machado|   35|350130|         35112|     ALTA SOROCABANA|2022-08-09|       32|           24915|        4110.0|         4|             88|          0|            null|                 null|
|Sudeste|    SP|           Alumínio|   35|350115|         35163|            SOROCABA|2022-08-09|       32|           18628|        1

In [21]:
covid_sp2 = covid.filter((f.col("estado") == "SP")&(f.col("codmun").isNull()))\
                  .orderBy(col("data").desc())

covid_sp2.show()

+-------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
| regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|
+-------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
|Sudeste|    SP|     null|   35|  null|          null|           null|2022-08-09|       32|        45919049|     5955410.0|      5097|         173418|         72|            null|                 null|
|Sudeste|    SP|     null|   35|  null|          null|           null|2022-08-08|       32|        45919049|     5950313.0|       840|         173346|          8|            null|             

In [22]:
covid_rj = covid.filter((f.col("estado") == "RJ")&(f.col("codmun").isNull()))\
                .orderBy(col("data").desc())

covid_rj.show()

+-------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
| regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|
+-------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
|Sudeste|    RJ|     null|   33|  null|          null|           null|2022-08-09|       32|        17264943|     2462761.0|      2365|          75089|         76|            null|                 null|
|Sudeste|    RJ|     null|   33|  null|          null|           null|2022-08-08|       32|        17264943|     2460396.0|      2753|          75013|         68|            null|             

In [23]:
 #covid_localidades = covid.filter((f.col("regiao") != "Brasil")&(f.col("municipio").isNotNull())&(f.col("casosNovos")<0))

covid_localidades = covid.filter((f.col("regiao") != "Brasil")\
                                 &(f.col("municipio").isNotNull())\
                                 &(f.col("codmun")== "110003")\
                                 &(f.col("casosNovos")!= 0))
covid_localidades.show()


+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
|regiao|estado|municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|
+------+------+---------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
| Norte|    RO|   Cabixi|   11|110003|         11006|       CONE SUL|2020-06-04|       23|            5312|           3.0|         3|              0|          0|            null|                 null|
| Norte|    RO|   Cabixi|   11|110003|         11006|       CONE SUL|2020-06-30|       27|            5312|           4.0|         1|              1|          1|            null|                 n

#### Comparando o total de casos novos quando o codigo do municipio for nulo X total de casos novos por municipio para a mesma data base, concluímos que o codigo do municipio igual a nulo não se refere a somatória do estado. Nesse caso, optamos por remover esses registros para evitar trabalhar com dados sujos.

In [24]:
covid_novo = covid.withColumn("casosNovos", when(f.col("casosNovos")<0, 0)\
                              .otherwise(f.col("casosNovos")))\
                  .withColumn("obitosNovos", when(f.col("obitosNovos")<0, 0)\
                              .otherwise(f.col("obitosNovos")))\
                  .filter((f.col("regiao") != "Brasil")&(f.col("municipio").isNotNull()))
covid_novo.show()

+------+------+--------------------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
|regiao|estado|           municipio|coduf|codmun|codRegiaoSaude|nomeRegiaoSaude|      data|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|
+------+------+--------------------+-----+------+--------------+---------------+----------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+
| Norte|    RO|Alta Floresta D'O...|   11|110001|         11005|   ZONA DA MATA|2020-03-27|       13|           22945|           0.0|         0|              0|          0|            null|                 null|
| Norte|    RO|Alta Floresta D'O...|   11|110001|         11005|   ZONA DA MATA|2020-03-28|       13|           22945|           0.0|         0|        

#### Foi realizada a soma de de total de casaos e obitos novos considerando somente o estado.

In [25]:
covid_total_estados = covid.withColumn("casosNovos", when(f.col("casosNovos")<0, 0)\
                                       .otherwise(f.col("casosNovos")))\
                           .withColumn("obitosNovos", when(f.col("obitosNovos")<0, 0)\
                                       .otherwise(f.col("obitosNovos")))\
                           .filter((f.col("municipio").isNull())&(f.col("regiao") != "Brasil"))\
                           .groupBy("estado" ,"coduf", "data")\
                           .agg(f.sum("casosNovos").alias("casosNovos_diarios_por_estado"),\
                                  sum("obitosNovos").alias("obitosNovos_diarios_por_estado"))
                                                               
covid_total_estados.show()

+------+-----+----------+-----------------------------+------------------------------+
|estado|coduf|      data|casosNovos_diarios_por_estado|obitosNovos_diarios_por_estado|
+------+-----+----------+-----------------------------+------------------------------+
|    RO|   11|2020-03-01|                            0|                             0|
|    RO|   11|2020-05-29|                          500|                             6|
|    AC|   12|2020-03-12|                            0|                             0|
|    AC|   12|2020-07-19|                          186|                             6|
|    AM|   13|2020-05-02|                          678|                            50|
|    PA|   15|2020-05-16|                         2150|                           108|
|    AP|   16|2020-05-19|                          574|                            18|
|    TO|   17|2020-04-05|                            6|                             0|
|    PI|   22|2020-05-11|                  

#### Foi realizado o joim dos dataframes covid_novo e covid_total_estados para melhor visualização

In [26]:
covid_total = covid_novo.join(covid_total_estados, ["data", "coduf","estado"], how='left')

covid_total.show()

+----------+-----+------+------+--------------------+------+--------------+---------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+-----------------------------+------------------------------+
|      data|coduf|estado|regiao|           municipio|codmun|codRegiaoSaude|nomeRegiaoSaude|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|casosNovos_diarios_por_estado|obitosNovos_diarios_por_estado|
+----------+-----+------+------+--------------------+------+--------------+---------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+-----------------------------+------------------------------+
|2020-04-03|   11|    RO| Norte|Alta Floresta D'O...|110001|         11005|   ZONA DA MATA|       14|           22945|           0.0|         0|              0|          0|         

In [27]:
municipios = covid_total.select("data", "regiao", "estado", "coduf", "municipio","casosNovos", \
                                "obitosNovos", "casosNovos_diarios_por_estado", "obitosNovos_diarios_por_estado")
                              
municipios.orderBy(col("data").desc()).show()

+----------+-------+------+-----+--------------------+----------+-----------+-----------------------------+------------------------------+
|      data| regiao|estado|coduf|           municipio|casosNovos|obitosNovos|casosNovos_diarios_por_estado|obitosNovos_diarios_por_estado|
+----------+-------+------+-----+--------------------+----------+-----------+-----------------------------+------------------------------+
|2022-08-09|Sudeste|    ES|   32|      Afonso Cláudio|         6|          0|                          651|                             7|
|2022-08-09|Sudeste|    ES|   32|Conceição do Castelo|         0|          0|                          651|                             7|
|2022-08-09|Sudeste|    ES|   32|        Águia Branca|         0|          0|                          651|                             7|
|2022-08-09|Sudeste|    ES|   32|  Água Doce do Norte|         2|          0|                          651|                             7|
|2022-08-09|Sudeste|    ES|

In [28]:
municipios.orderBy(col("data").desc(), col("casosNovos_diarios_por_estado").desc()).show()

+----------+-------+------+-----+--------------------+----------+-----------+-----------------------------+------------------------------+
|      data| regiao|estado|coduf|           municipio|casosNovos|obitosNovos|casosNovos_diarios_por_estado|obitosNovos_diarios_por_estado|
+----------+-------+------+-----+--------------------+----------+-----------+-----------------------------+------------------------------+
|2022-08-09|Sudeste|    SP|   35|           Americana|         0|          1|                         5097|                            72|
|2022-08-09|Sudeste|    SP|   35|              Altair|         1|          0|                         5097|                            72|
|2022-08-09|Sudeste|    SP|   35|         Alvinlândia|         0|          0|                         5097|                            72|
|2022-08-09|Sudeste|    SP|   35|Águas de Santa Bá...|         0|          0|                         5097|                            72|
|2022-08-09|Sudeste|    SP|

In [29]:
municipios.select("data", "estado", "coduf", "casosNovos_diarios_por_estado", "obitosNovos_diarios_por_estado").distinct()\
          .orderBy(col("data").desc(), col("obitosNovos_diarios_por_estado").desc()).show()

+----------+------+-----+-----------------------------+------------------------------+
|      data|estado|coduf|casosNovos_diarios_por_estado|obitosNovos_diarios_por_estado|
+----------+------+-----+-----------------------------+------------------------------+
|2022-08-09|    RJ|   33|                         2365|                            76|
|2022-08-09|    SP|   35|                         5097|                            72|
|2022-08-09|    PR|   41|                         4168|                            40|
|2022-08-09|    MG|   31|                         2095|                            30|
|2022-08-09|    MS|   50|                          539|                            23|
|2022-08-09|    CE|   23|                          819|                            22|
|2022-08-09|    RS|   43|                         3402|                            21|
|2022-08-09|    BA|   29|                         1597|                            19|
|2022-08-09|    GO|   52|                  

#### Foram explorados valor máximo, mínimo e média de casos e obitos novos por município.

In [30]:
df_casos = municipios.groupBy("data","estado", "coduf", "regiao")\
                     .agg(round(avg("casosNovos"), 2).alias("media_casos_Novos_estado"),\
                          round(max("casosNovos"), 2).alias("max_casos_Novos_estado"),\
                          round(min("casosNovos"), 2).alias("min_casos_Novos_estado"))\
                     .sort(col("media_casos_Novos_estado").desc())
              

df_obitos= municipios.groupBy("data","estado", "coduf", "regiao")\
                     .agg(round(avg("obitosNovos"), 2).alias("media_obitos_Novos_estado"),\
                          round(max("obitosNovos"), 2).alias("max_obitos_Novos_estado"),\
                          round(min("obitosNovos"), 2).alias("min_obitos_Novos_estado"))\
                     .sort(col("media_obitos_Novos_estado").desc())
              

df_casos.show()
df_obitos.show()

+----------+------+-----+------------+------------------------+----------------------+----------------------+
|      data|estado|coduf|      regiao|media_casos_Novos_estado|max_casos_Novos_estado|min_casos_Novos_estado|
+----------+------+-----+------------+------------------------+----------------------+----------------------+
|2022-01-31|    DF|   53|Centro-Oeste|                 11508.0|                 11508|                 11508|
|2022-01-25|    DF|   53|Centro-Oeste|                 10697.0|                 10697|                 10697|
|2022-06-20|    DF|   53|Centro-Oeste|                  9007.0|                  9007|                  9007|
|2022-02-07|    DF|   53|Centro-Oeste|                  8773.0|                  8773|                  8773|
|2022-06-13|    DF|   53|Centro-Oeste|                  8711.0|                  8711|                  8711|
|2022-02-03|    DF|   53|Centro-Oeste|                  7086.0|                  7086|                  7086|
|2022-06-1

####Foi realizada a soma do valor total de casos novos e óbitos por região considerando o tempo total.

In [31]:
regiao_ob = regiao.drop("data")\
                  .groupby("regiao")\
                  .agg(sum("obitosNovos").alias("total_obitos"))\
                  .filter(column("total_obitos") >= 0)\
                  .sort(col("total_obitos").desc())\
                  .withColumnRenamed("regiao","Regioes")

regiao_cs = regiao.drop("data")\
                  .groupby("regiao")\
                  .agg(sum("casosNovos").alias("total_casos"))\
                  .filter(column("total_casos") >= 0)\
                  .sort(col("total_casos").desc())
regiao_ob.show()
regiao_cs.show()

+------------+------------+
|     Regioes|total_obitos|
+------------+------------+
|     Sudeste|      736392|
|    Nordeste|      319672|
|         Sul|      224856|
|Centro-Oeste|      140066|
|       Norte|      125026|
+------------+------------+

+------------+-----------+
|      regiao|total_casos|
+------------+-----------+
|     Sudeste|   28775294|
|    Nordeste|   15290244|
|         Sul|   14903626|
|Centro-Oeste|    8239030|
|       Norte|    6248776|
+------------+-----------+



In [32]:
poluicao_ar = spark.read.format('csv')\
                        .options(header='true',\
                                 inferschema='true')\
                   .load(f'{caminho_local}/poluicao.csv',header=True)

In [33]:
print("Dados de 2020")
poluicao = poluicao_ar.sort(col("Total_Poluicao_do_ar").desc())

poluicao.show()

Dados de 2020
+------------+--------------------+
|      Regiao|Total_Poluicao_do_ar|
+------------+--------------------+
|    Nordeste|               126.0|
|     Sudeste|                92.0|
|       Norte|                53.0|
|         Sul|                46.0|
|Centro-Oeste|                36.0|
+------------+--------------------+



In [34]:
desemprego = spark.read.format('csv')\
                       .options(header='true',\
                                inferschema='true')\
                       .load(f'{caminho_local}/desemprego.csv',header=True)

desemprego.show(50)


+-------------------+----------------+----------------+
|             Estado|1_trimestre_2020|2_trimestre_2022|
+-------------------+----------------+----------------+
|           Rondônia|              76|              52|
|               Acre|              47|              45|
|           Amazonas|             279|             203|
|            Roraima|              42|              16|
|               Pará|             417|             377|
|              Amapá|              67|              48|
|          Tocantins|              84|              43|
|           Maranhão|             431|             299|
|              Piauí|             202|             135|
|              Ceará|             513|             412|
|Rio Grande do Norte|             244|             188|
|            Paraíba|             232|             203|
|         Pernambuco|             611|             578|
|            Alagoas|             204|             149|
|            Sergipe|             171|          

#### Foram alterados os valores da coluna Estado no dataframe de desemprego, de nome do estado para a sigla do estado para futura comparação com os dados de Covid.

In [35]:
desemprego = desemprego.withColumn('Estado',\
                                   regexp_replace('Estado', 'Rondônia', 'RO'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Acre', 'AC'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Amazonas', 'AM'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Roraima', 'RR'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Pará', 'PA'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Amapá', 'AP'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Tocantins', 'TO'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Maranhão', 'MA'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Piauí', 'PI'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Ceará', 'CE'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Rio Grande do Norte', 'RN'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Paraíba', 'PB'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Pernambuco', 'PE'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Alagoas', 'AL'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Sergipe', 'SE'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Bahia', 'BA'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Minas Gerais', 'MG'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Espírito Santo', 'ES'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Rio de Janeiro', 'RJ'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'São Paulo', 'SP'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Paraná', 'PR'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Santa Catarina', 'SC'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Rio Grande do Sul', 'RS' ))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Mato Grosso do Sul', 'MS'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Mato Grosso', 'MT'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Goiás', 'GO'))\
                       .withColumn('Estado',\
                                   regexp_replace('Estado', 'Distrito Federal', 'DF'))\

desemprego.show(27)

+------+----------------+----------------+
|Estado|1_trimestre_2020|2_trimestre_2022|
+------+----------------+----------------+
|    RO|              76|              52|
|    AC|              47|              45|
|    AM|             279|             203|
|    RR|              42|              16|
|    PA|             417|             377|
|    AP|              67|              48|
|    TO|              84|              43|
|    MA|             431|             299|
|    PI|             202|             135|
|    CE|             513|             412|
|    RN|             244|             188|
|    PB|             232|             203|
|    PE|             611|             578|
|    AL|             204|             149|
|    SE|             171|             140|
|    BA|            1333|            1104|
|    MG|            1320|             828|
|    ES|             243|             174|
|    RJ|            1313|            1130|
|    SP|            3151|            2404|
|    PR|   

In [36]:
print("Pessoas desempregadas na semana da pesquisa(Mil pessoas)")
desemprego20 = desemprego.sort(col("1_trimestre_2020").desc())

desemprego20.show()

Pessoas desempregadas na semana da pesquisa(Mil pessoas)
+------+----------------+----------------+
|Estado|1_trimestre_2020|2_trimestre_2022|
+------+----------------+----------------+
|    SP|            3151|            2404|
|    BA|            1333|            1104|
|    MG|            1320|             828|
|    RJ|            1313|            1130|
|    PE|             611|             578|
|    RS|             529|             387|
|    CE|             513|             412|
|    PR|             487|             374|
|    GO|             434|             270|
|    MA|             431|             299|
|    PA|             417|             377|
|    AM|             279|             203|
|    RN|             244|             188|
|    ES|             243|             174|
|    PB|             232|             203|
|    DF|             228|             205|
|    SC|             223|             160|
|    AL|             204|             149|
|    PI|             202|             13

In [37]:
print("Pessoas desempregadas na semana da pesquisa(Mil pessoas)")
desemprego22 = desemprego.sort(col("2_trimestre_2022").desc())

desemprego22.show()

Pessoas desempregadas na semana da pesquisa(Mil pessoas)
+------+----------------+----------------+
|Estado|1_trimestre_2020|2_trimestre_2022|
+------+----------------+----------------+
|    SP|            3151|            2404|
|    RJ|            1313|            1130|
|    BA|            1333|            1104|
|    MG|            1320|             828|
|    PE|             611|             578|
|    CE|             513|             412|
|    RS|             529|             387|
|    PA|             417|             377|
|    PR|             487|             374|
|    MA|             431|             299|
|    GO|             434|             270|
|    DF|             228|             205|
|    AM|             279|             203|
|    PB|             232|             203|
|    RN|             244|             188|
|    ES|             243|             174|
|    SC|             223|             160|
|    AL|             204|             149|
|    SE|             171|             14

####Foi realizada a soma do valor total de casos novos e óbitos por região considerando o tempo total.

In [38]:
estado_total_ob = estados.drop("data")\
                         .groupby("estado")\
                         .agg(sum("obitosNovos").alias("total_obitos"))\
                         .filter(column("total_obitos") >= 0)\
                         .sort(col("total_obitos").desc())\
                         .withColumnRenamed("estado","Estados")
                
estado_total_ob.show()

+-------+------------+
|Estados|total_obitos|
+-------+------------+
|     SP|      196415|
|     RJ|       88566|
|     MG|       66390|
|     PR|       46868|
|     RS|       42446|
|     CE|       35142|
|     BA|       34321|
|     GO|       28902|
|     PE|       28701|
|     PA|       24301|
|     SC|       23378|
|     ES|       17569|
|     AM|       17507|
|     MT|       16716|
|     MA|       13969|
|     DF|       13292|
|     PB|       12187|
|     MS|       11123|
|     RN|       10193|
|     PI|        9237|
+-------+------------+
only showing top 20 rows



In [39]:
estado_total_cs = estados.drop("data")\
                         .groupby("estado")\
                         .agg(sum("casosNovos").alias("total_casos"))\
                         .filter(column("total_casos") >= 0)\
                         .sort(col("total_casos").desc())

#soma = estado_total_cs.groupBy().agg(f.sum(f.col("total_casos")))

# estado_total_cs =estado_total_cs.withColumn('total_br', sum("total_casos"))
                
# estado_total_cs.show()

In [40]:
covid_total2 = covid_total.join(estado_total_cs, ["estado"], how='left')\
                          .join(estado_total_ob, covid_total.estado ==  estado_total_ob.Estados, how='left')\
                          .drop(estado_total_ob.Estados)

covid_total2.show()

+------+----------+-----+------+--------------------+------+--------------+---------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+-----------------------------+------------------------------+-----------+------------+
|estado|      data|coduf|regiao|           municipio|codmun|codRegiaoSaude|nomeRegiaoSaude|semanaEpi|populacaoTCU2019|casosAcumulado|casosNovos|obitosAcumulado|obitosNovos|Recuperadosnovos|emAcompanhamentoNovos|casosNovos_diarios_por_estado|obitosNovos_diarios_por_estado|total_casos|total_obitos|
+------+----------+-----+------+--------------------+------+--------------+---------------+---------+----------------+--------------+----------+---------------+-----------+----------------+---------------------+-----------------------------+------------------------------+-----------+------------+
|    RO|2020-04-03|   11| Norte|Alta Floresta D'O...|110001|         11005|   ZONA DA MATA|       14|     

#### Demostração dos dados em gráfico.

In [41]:
graf = estado_total_ob.join(desemprego20, estado_total_ob.Estados == desemprego.Estado, "inner")\
                      .drop("Estado")
graf = graf.toPandas()

In [42]:
to_plot = [v for v in list(graf.columns)]

fig_graf_desemprego_covid = px.bar(graf, x=graf.Estados, y=to_plot,\
                                   barmode="group",\
                                   text_auto='.2s',\
                                   title="Obitos por Covid e Desemprego")
             
fig_graf_desemprego_covid.show()


graf_desemprego_covid = px.bar(graf, x='Estados', y="total_obitos",\
                               barmode="group",\
                               color="total_obitos",\
                               title="Obitos por Covid")
graf_desemprego_covidx = px.bar(graf, x='Estados', y="1_trimestre_2020",\
                                barmode="group",\
                                color="1_trimestre_2020",\
                                title="Desemprego no primeiro trimestre de 2020")
graf_desemprego_covidxx = px.bar(graf, x='Estados', y="2_trimestre_2022",\
                                 barmode="group",\
                                 color="2_trimestre_2022",\
                                 title="Desemprego no segundo trimestre de 2022")
graf_desemprego_covid.show()
graf_desemprego_covidx.show()
graf_desemprego_covidxx.show()

In [43]:
graf2 = regiao_ob.join(poluicao_ar, regiao_ob.Regioes == poluicao_ar.Regiao, "inner")\
                 .drop("Regiao")
graf2 = graf2.toPandas()

In [44]:
to_plot_2= [v for v in list(graf2.columns)]

fig_graf_poluicao_covid = px.bar(graf2, x=graf2.Regioes, y=to_plot_2,\
                                 barmode="group",\
                                 text_auto='.2s',\
                                 title="Obitos por Covid e Poluição")
fig_graf_poluicao_covid.show()

graf_poluicao_covid = px.bar(graf2, x='Regioes', y="total_obitos",\
                             barmode="group",\
                             color="total_obitos",\
                             title="Obitos por covid")
graf_poluicao_covidx = px.bar(graf2, x='Regioes', y="Total_Poluicao_do_ar",\
                              barmode="group",\
                              color="Total_Poluicao_do_ar",\
                              title='Poluição do ar')

graf_poluicao_covid.show()
graf_poluicao_covidx.show()

#### Importação da tabela de educação no Brasil referente aos anos de 2016,2017 e 2018, devido não encontrar dados mais rescentes




In [45]:
educacao = spark.read.format('csv')\
                     .options(header='true',\
                              inferschema='true')\
                .load(f'{caminho_local}/educacao.csv',header=True)

In [46]:
educacao.show()

+--------------------+--------------------+--------------------+--------------------+---------+----------------------+---------------+----+----+----+
|           Indicador|   Nível_Territorial|Abertura_Territorial|Variável_de_abertura|Categoria|Variável_de_abertura_2|    Categoria_2|2016|2017|2018|
+--------------------+--------------------+--------------------+--------------------+---------+----------------------+---------------+----+----+----+
|Taxa de analfabet...|Unidade da Federação|            Rondônia|                Sexo|    Total|  Grupos de idade -...|15 anos ou mais| 6.6| 7.2| 6.5|
|Taxa de analfabet...|Unidade da Federação|            Rondônia|                Sexo|    Total|  Grupos de idade -...|18 anos ou mais| 7.2| 7.6| 6.9|
|Taxa de analfabet...|Unidade da Federação|            Rondônia|                Sexo|    Total|  Grupos de idade -...|25 anos ou mais| 8.4| 8.9| 8.3|
|Taxa de analfabet...|Unidade da Federação|            Rondônia|                Sexo|    Total|  Gru

#### Foram alterados os valores da coluna Abertura_Territorial no dataframe de Educação, de nome do estado para a sigla do estado para futura comparação com os dados de Covid.

In [47]:
educacao = educacao.withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Rondônia', 'RO'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Acre', 'AC'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Amazonas', 'AM'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Roraima', 'RR'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Pará', 'PA'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Amapá', 'AP'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Tocantins', 'TO'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Maranhão', 'MA'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Piauí', 'PI'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Ceará', 'CE'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Rio Grande do Norte', 'RN'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Paraíba', 'PB'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Pernambuco', 'PE'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Alagoas', 'AL'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Sergipe', 'SE'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Bahia', 'BA'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Minas Gerais', 'MG'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Espírito Santo', 'ES'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Rio de Janeiro', 'RJ'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'São Paulo', 'SP'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Paraná', 'PR'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Santa Catarina', 'SC'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Rio Grande do Sul', 'RS' ))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Mato Grosso do Sul', 'MS'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Mato Grosso', 'MT'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Goiás', 'GO'))\
                       .withColumn('Abertura_Territorial',\
                                   regexp_replace('Abertura_Territorial', 'Distrito Federal', 'DF'))\

educacao.show(27)

+--------------------+--------------------+--------------------+--------------------+---------+----------------------+---------------+----+----+----+
|           Indicador|   Nível_Territorial|Abertura_Territorial|Variável_de_abertura|Categoria|Variável_de_abertura_2|    Categoria_2|2016|2017|2018|
+--------------------+--------------------+--------------------+--------------------+---------+----------------------+---------------+----+----+----+
|Taxa de analfabet...|Unidade da Federação|                  RO|                Sexo|    Total|  Grupos de idade -...|15 anos ou mais| 6.6| 7.2| 6.5|
|Taxa de analfabet...|Unidade da Federação|                  RO|                Sexo|    Total|  Grupos de idade -...|18 anos ou mais| 7.2| 7.6| 6.9|
|Taxa de analfabet...|Unidade da Federação|                  RO|                Sexo|    Total|  Grupos de idade -...|25 anos ou mais| 8.4| 8.9| 8.3|
|Taxa de analfabet...|Unidade da Federação|                  RO|                Sexo|    Total|  Gru

#### Foi feita a separaçao da coluna categoria_2 referentes ao ano de 2018 de acordo com a idade para melhor visualização dos dados.

In [48]:
educacao.createOrReplaceTempView("new_df")

idade_15_ou_mais = spark.sql("select Abertura_Territorial,  Categoria_2, `2018` \
                              from new_df\
                              where Categoria_2 like '%15%'")\
                        .orderBy(col('2018').desc())

idade_15_ou_mais.show(100)

+--------------------+---------------+----+
|Abertura_Territorial|    Categoria_2|2018|
+--------------------+---------------+----+
|                  AL|15 anos ou mais|17.2|
|                  PI|15 anos ou mais|16.6|
|                  MA|15 anos ou mais|16.3|
|                  PB|15 anos ou mais|16.1|
|                  SE|15 anos ou mais|13.9|
|                  CE|15 anos ou mais|13.3|
|                  RN|15 anos ou mais|12.9|
|                  BA|15 anos ou mais|12.7|
|                  AC|15 anos ou mais|12.1|
|                  PE|15 anos ou mais|11.9|
|                  TO|15 anos ou mais|10.0|
|                  PA|15 anos ou mais| 8.8|
|                  MT|15 anos ou mais| 7.1|
|                  RO|15 anos ou mais| 6.5|
|                  AP|15 anos ou mais| 6.1|
|                  RR|15 anos ou mais| 6.0|
|                  MG|15 anos ou mais| 5.8|
|                  AM|15 anos ou mais| 5.8|
|                  GO|15 anos ou mais| 5.7|
|                  ES|15 anos ou

In [49]:
educacao.createOrReplaceTempView("new_df")

idade_18_ou_mais = spark.sql("select Abertura_Territorial,  Categoria_2, `2018` \
                              from new_df\
                              where Categoria_2 like '%18%'")\
                        .orderBy(col('2018').desc())

idade_18_ou_mais.show(100)

+--------------------+---------------+----+
|Abertura_Territorial|    Categoria_2|2018|
+--------------------+---------------+----+
|                  AL|18 anos ou mais|18.5|
|                  PI|18 anos ou mais|17.7|
|                  MA|18 anos ou mais|17.6|
|                  PB|18 anos ou mais|17.2|
|                  SE|18 anos ou mais|14.8|
|                  CE|18 anos ou mais|14.2|
|                  RN|18 anos ou mais|13.7|
|                  BA|18 anos ou mais|13.5|
|                  AC|18 anos ou mais|13.2|
|                  PE|18 anos ou mais|12.6|
|                  TO|18 anos ou mais|10.7|
|                  PA|18 anos ou mais| 9.4|
|                  MT|18 anos ou mais| 7.5|
|                  RO|18 anos ou mais| 6.9|
|                  RR|18 anos ou mais| 6.5|
|                  AP|18 anos ou mais| 6.5|
|                  AM|18 anos ou mais| 6.3|
|                  MG|18 anos ou mais| 6.2|
|                  GO|18 anos ou mais| 6.1|
|                  ES|18 anos ou

In [50]:
educacao.createOrReplaceTempView("new_df")

idade_25_ou_mais = spark.sql("select Abertura_Territorial,  Categoria_2, `2018` \
                              from new_df\
                              where Categoria_2 like '%25%'")\
                        .orderBy(col('2018').desc())

idade_25_ou_mais.show(100)

+--------------------+---------------+----+
|Abertura_Territorial|    Categoria_2|2018|
+--------------------+---------------+----+
|                  AL|25 anos ou mais|21.5|
|                  MA|25 anos ou mais|21.0|
|                  PI|25 anos ou mais|20.7|
|                  PB|25 anos ou mais|19.9|
|                  SE|25 anos ou mais|17.5|
|                  CE|25 anos ou mais|16.6|
|                  RN|25 anos ou mais|16.1|
|                  AC|25 anos ou mais|15.9|
|                  BA|25 anos ou mais|15.6|
|                  PE|25 anos ou mais|14.6|
|                  TO|25 anos ou mais|12.4|
|                  PA|25 anos ou mais|11.2|
|                  MT|25 anos ou mais| 8.7|
|                  RO|25 anos ou mais| 8.3|
|                  AP|25 anos ou mais| 8.2|
|                  RR|25 anos ou mais| 7.9|
|                  AM|25 anos ou mais| 7.6|
|                  GO|25 anos ou mais| 7.0|
|                  MG|25 anos ou mais| 7.0|
|                  ES|25 anos ou

In [51]:
educacao.createOrReplaceTempView("new_df")

idade_40_ou_mais = spark.sql("select Abertura_Territorial,  Categoria_2, `2018` \
                              from new_df\
                              where Categoria_2 like '%40%'")\
                        .orderBy(col('2018').desc())

idade_40_ou_mais.show(100)

+--------------------+---------------+----+
|Abertura_Territorial|    Categoria_2|2018|
+--------------------+---------------+----+
|                  MA|40 anos ou mais|30.5|
|                  AL|40 anos ou mais|29.2|
|                  PI|40 anos ou mais|28.4|
|                  PB|40 anos ou mais|26.6|
|                  SE|40 anos ou mais|24.8|
|                  CE|40 anos ou mais|24.1|
|                  AC|40 anos ou mais|23.4|
|                  RN|40 anos ou mais|22.5|
|                  BA|40 anos ou mais|22.3|
|                  PE|40 anos ou mais|20.1|
|                  TO|40 anos ou mais|18.5|
|                  PA|40 anos ou mais|16.9|
|                  AP|40 anos ou mais|13.4|
|                  RO|40 anos ou mais|13.1|
|                  RR|40 anos ou mais|12.9|
|                  MT|40 anos ou mais|12.8|
|                  AM|40 anos ou mais|11.2|
|                  GO|40 anos ou mais|10.3|
|                  MG|40 anos ou mais|10.0|
|                  ES|40 anos ou

In [52]:
educacao.createOrReplaceTempView("new_df")

idade_60_ou_mais = spark.sql("select Abertura_Territorial,  Categoria_2, `2018`\
                              from new_df\
                              where Categoria_2 like '%60%'")\
                        .orderBy(col('2018').desc())
idade_60_ou_mais.show(100)




+--------------------+---------------+----+
|Abertura_Territorial|    Categoria_2|2018|
+--------------------+---------------+----+
|                  MA|60 anos ou mais|45.5|
|                  PI|60 anos ou mais|42.7|
|                  AL|60 anos ou mais|41.1|
|                  SE|60 anos ou mais|38.6|
|                  PB|60 anos ou mais|37.8|
|                  CE|60 anos ou mais|36.9|
|                  AC|60 anos ou mais|36.5|
|                  BA|60 anos ou mais|35.3|
|                  RN|60 anos ou mais|34.8|
|                  TO|60 anos ou mais|32.5|
|                  PE|60 anos ou mais|30.2|
|                  PA|60 anos ou mais|27.9|
|                  RO|60 anos ou mais|27.5|
|                  AP|60 anos ou mais|27.4|
|                  RR|60 anos ou mais|25.9|
|                  MT|60 anos ou mais|24.2|
|                  AM|60 anos ou mais|19.9|
|                  GO|60 anos ou mais|18.9|
|                  ES|60 anos ou mais|18.9|
|                  MG|60 anos ou

#### Demostração dos dados em gráfico de pessoas de 60 anos ou mais.

In [53]:
graf_educacao = estado_total_cs.join(idade_60_ou_mais, estado_total_cs.estado == idade_60_ou_mais.Abertura_Territorial, "inner")\
                      .drop("Abertura_Territorial")\
                      .drop("Categoria_2")
                      
graf_educacao = graf_educacao.toPandas()

In [54]:
to_plot_educacao = [v for v in list(graf_educacao.columns)]

fig_graf_educacao_covid = px.bar(graf_educacao, x=graf_educacao.estado, y=to_plot_educacao,\
                                   barmode="stack",\
                                   text_auto='.2s',\
                                   title="Casos novos de Covid e IDH")
             
fig_graf_educacao_covid.show()


graf_educacao_covid_cs = px.bar(graf_educacao, x='estado', y="total_casos",\
                               barmode="group",\
                               color="total_casos",\
                               title="Casos novos de Covid")
graf_educacao_covid= px.bar(graf_educacao, x='estado', y="2018",\
                                barmode="group",\
                                color="2018",\
                                title="Taxa de analfabetismo em 2018")

graf_educacao_covid_cs.show()
graf_educacao_covid.show()

#### Importação da tabela de IDHM no Brasil referente ao ano de 2017, devido não encontrar dados mais rescentes.

In [55]:
idh = spark.read.format('csv')\
                     .options(header='true',\
                              inferschema='true')\
                .load(f'{caminho_local}/idh.csv',header=True)
idh.show()

+------------------+------------+-----+------------------+----------+---------------------+-------------+------------------------+----------------+
|  Territorialidade|Posição_IDHM| IDHM|Posição_IDHM_Renda|IDHM_Renda|Posição_IDHM_Educação|IDHM_Educação|Posição_IDHM_Longevidade|IDHM_Longevidade|
+------------------+------------+-----+------------------+----------+---------------------+-------------+------------------------+----------------+
|         São Paulo|           2|0.826|                 5|     0.854|                    1|        0.828|                       2|           0.796|
|  Distrito Federal|           1| 0.85|                 1|      0.89|                    2|        0.804|                       1|           0.859|
|    Santa Catarina|           3|0.808|                 3|     0.866|                    3|        0.779|                       4|           0.783|
|           Roraima|          11|0.752|                22|     0.781|                    4|        0.771|       

#### Foram alterados os valores da coluna Territorialidade no dataframe IDHM, de nome do estado para a sigla do estado para futura comparação com os dados de Covid.

In [56]:
idh = idh.withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Rondônia', 'RO'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Acre', 'AC'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Amazonas', 'AM'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Roraima', 'RR'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Pará', 'PA'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Amapá', 'AP'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Tocantins', 'TO'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Maranhão', 'MA'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Piauí', 'PI'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Ceará', 'CE'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Rio Grande do Norte', 'RN'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Paraíba', 'PB'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Pernambuco', 'PE'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Alagoas', 'AL'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Sergipe', 'SE'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Bahia', 'BA'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Minas Gerais', 'MG'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Espírito Santo', 'ES'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Rio de Janeiro', 'RJ'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'São Paulo', 'SP'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Paraná', 'PR'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Santa Catarina', 'SC'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Rio Grande do Sul', 'RS' ))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Mato Grosso do Sul', 'MS'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Mato Grosso', 'MT'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Goiás', 'GO'))\
                       .withColumn('Territorialidade',\
                                   regexp_replace('Territorialidade', 'Distrito Federal', 'DF'))\

idh.show(27)

+----------------+------------+-----+------------------+----------+---------------------+-------------+------------------------+----------------+
|Territorialidade|Posição_IDHM| IDHM|Posição_IDHM_Renda|IDHM_Renda|Posição_IDHM_Educação|IDHM_Educação|Posição_IDHM_Longevidade|IDHM_Longevidade|
+----------------+------------+-----+------------------+----------+---------------------+-------------+------------------------+----------------+
|              SP|           2|0.826|                 5|     0.854|                    1|        0.828|                       2|           0.796|
|              DF|           1| 0.85|                 1|      0.89|                    2|        0.804|                       1|           0.859|
|              SC|           3|0.808|                 3|     0.866|                    3|        0.779|                       4|           0.783|
|              RR|          11|0.752|                22|     0.781|                    4|        0.771|                     

#### Foi feita a separaçao do dataframe de acordo com o IDHM considerado para melhor visualização dos dados.

In [57]:
idhm = idh.select("Territorialidade", "Posição_IDHM", "IDHM")\
          .orderBy(col("IDHM").asc())

idhm.show(27)

+----------------+------------+-----+
|Territorialidade|Posição_IDHM| IDHM|
+----------------+------------+-----+
|              AL|          26|0.683|
|              MA|          25|0.687|
|              PI|          24|0.697|
|              PA|          23|0.698|
|              SE|          22|0.702|
|              BA|          21|0.714|
|              AC|          20|0.719|
|              PB|          19|0.722|
|              RO|          18|0.725|
|              PE|          17|0.727|
|              RN|          16|0.731|
|              AM|          15|0.733|
|              CE|          14|0.735|
|              AP|          13| 0.74|
|              TO|          12|0.743|
|              RR|          11|0.752|
|              MS|          10|0.766|
|              GO|           9|0.769|
|              ES|           8|0.772|
|              MT|           7|0.774|
|              MG|           6|0.787|
|              RS|           6|0.787|
|              PR|           5|0.792|
|           

In [58]:
idhm_renda = idh.select("Territorialidade", "Posição_IDHM_Renda", "IDHM_Renda")\
          .orderBy(col("IDHM_Renda").asc())

idhm_renda.show(27)

+----------------+------------------+----------+
|Territorialidade|Posição_IDHM_Renda|IDHM_Renda|
+----------------+------------------+----------+
|              MA|                25|     0.764|
|              PI|                24|     0.771|
|              RO|                23|     0.776|
|              RR|                22|     0.781|
|              AL|                21|     0.783|
|              AM|                20|     0.786|
|              PA|                19|     0.788|
|              SE|                18|     0.799|
|              PB|                17|     0.809|
|              TO|                16|     0.811|
|              BA|                15|     0.812|
|              CE|                14|     0.818|
|              AP|                13|      0.82|
|              AC|                12|     0.821|
|              PE|                12|     0.821|
|              GO|                11|     0.822|
|              MT|                10|     0.825|
|              PR|  

In [59]:
idhm_educacao = idh.select("Territorialidade", "Posição_IDHM_Educação", "IDHM_Educação")\
          .orderBy(col("IDHM_Educação").asc())

idhm_educacao.show(27)

+----------------+---------------------+-------------+
|Territorialidade|Posição_IDHM_Educação|IDHM_Educação|
+----------------+---------------------+-------------+
|              AL|                   25|        0.636|
|              SE|                   24|         0.64|
|              BA|                   23|        0.654|
|              PA|                   22|        0.661|
|              PI|                   21|        0.666|
|              PB|                   20|        0.671|
|              RN|                   19|        0.677|
|              MA|                   18|        0.682|
|              AC|                   18|        0.682|
|              PE|                   17|        0.685|
|              RO|                   16|        0.703|
|              MS|                   15|         0.71|
|              AP|                   15|         0.71|
|              CE|                   14|        0.717|
|              TO|                   13|        0.727|
|         

In [60]:
idhm_longevidade = idh.select("Territorialidade", "Posição_IDHM_Longevidade", "IDHM_Longevidade")\
          .orderBy(col("IDHM_Longevidade").asc())

idhm_longevidade.show(27)

+----------------+------------------------+----------------+
|Territorialidade|Posição_IDHM_Longevidade|IDHM_Longevidade|
+----------------+------------------------+----------------+
|              MA|                      26|           0.623|
|              AL|                      25|           0.639|
|              PA|                      24|           0.654|
|              PI|                      23|            0.66|
|              AC|                      22|           0.664|
|              CE|                      21|           0.676|
|              SE|                      20|           0.677|
|              RN|                      19|            0.68|
|              AM|                      18|           0.682|
|              PE|                      18|           0.682|
|              BA|                      17|           0.685|
|              PB|                      16|           0.694|
|              AP|                      15|           0.695|
|              TO|      

#### Demostração dos dados em gráfico.

In [61]:
graf_idh = estado_total_cs.join(idh, estado_total_cs.estado == idh.Territorialidade, "inner")\
                      .drop("Territorialidade")
graf_idh = graf_idh.toPandas()

In [62]:
to_plot_idh = [v for v in list(graf_idh.columns)]

fig_graf_idh_covid = px.bar(graf_idh, x=graf_idh.estado, y=to_plot_idh,\
                                   barmode="stack",\
                                   text_auto='.2s',\
                                   title="Casos novos de Covid e IDH")
             
fig_graf_idh_covid.show()


graf_idh_covid_obito = px.bar(graf_idh, x='estado', y="total_casos",\
                               barmode="group",\
                               color="total_casos",\
                               title="Casos novos de  Covid")
graf_idh_covid= px.bar(graf_idh, x='estado', y="IDHM",\
                                barmode="group",\
                                color="IDHM",\
                                title="IDHM")
graf_idh_covid_renda = px.bar(graf_idh, x='estado', y="IDHM_Renda",\
                                 barmode="group",\
                                 color="IDHM_Renda",\
                                 title="IDHM Renda")
graf_idh_covid_educacao = px.bar(graf_idh, x='estado', y="IDHM_Educação",\
                                 barmode="group",\
                                 color="IDHM_Educação",\
                                 title="IDHM Educação")
graf_idh_covid_educacao_longevidade = px.bar(graf_idh, x='estado', y="IDHM_Longevidade",\
                                 barmode="group",\
                                 color="IDHM_Longevidade",\
                                 title="IDHM Longevidade")
graf_idh_covid_obito.show()
graf_idh_covid.show()
graf_idh_covid_renda.show()
graf_idh_covid_educacao.show()
graf_idh_covid_educacao_longevidade.show()